# Querying graph databases with Sparql
This notebook contains examples and some challenges on using Sparql to query graph databases. These examples access the Wikidata endpoint. 

You can also [use the Wikidata Query Servce](https://query.wikidata.org/), which has tooltips for node and edge labels.

In [ ]:
import pandas as pd

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint = "https://query.wikidata.org/sparql"

In [ ]:
def sparql_pandas(query: str, endpoint: str):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    
    Arguments:
        query (str): a Sparql query written into a string
        endpoint (str): the url of the service to query, including https://
    
    Returns:
        pd.DataFrame: query response formed into a table
    """
    sparql = SPARQLWrapper(endpoint, agent="Sparql Wrapper in Jupyter notebook")  
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    result = sparql.query().convert()
    return pd.json_normalize(result["results"]["bindings"])

## Example
This query finds 20 American journalists in no particular order

The SERVICE line is a wikidata-specific thing, not part of regular Sparql. It specifies the language used in the labels. 

The labels are requested by the addition of 'Label' after the variable in the select clause. This makes sure you get human-readable results rather than unique ids.

In [ ]:
journalists = """
    SELECT ?personLabel
        WHERE {
            ?person 	wdt:P106 	wd:Q1930187 .
            ?person 	wdt:P27 	wd:Q30 .
            SERVICE wikibase:label { 
                bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                }
        }
    LIMIT 20
"""

sparql_pandas(journalists, endpoint)

## Challenge 1
Find all paintings containing cats

See my [sample answer](https://w.wiki/xtS)

In [ ]:
cats1 = """
    SELECT 
        WHERE {
           
            SERVICE wikibase:label { 
                bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                }
        }
"""

sparql_pandas(cats1, endpoint)

## Challenge 2
Find all paintings containing cats
That were painted in the 19th century

See my [sample answer](https://w.wiki/xta)

In [ ]:
cats2 = """
    SELECT 
        WHERE {
           
            SERVICE wikibase:label { 
                bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                }
        }
"""

sparql_pandas(cats2, endpoint)

## Challenge 3
Find all paintings containing cats
That were painted in the 19th century
By artists who were US citizens

See my [sample answer](https://w.wiki/xt$)

In [ ]:
cats3 = """
    SELECT 
        WHERE {
           
            SERVICE wikibase:label { 
                bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                }
        }
"""

sparql_pandas(cats3, endpoint)

## Challenge 4
Find all paintings containing cats
That were painted in the 19th century
By artists who were US citizens
Who have also painted dogs

See my [sample answer](https://w.wiki/xu4)

In [ ]:
cats4 = """
    SELECT 
        WHERE {
           
            SERVICE wikibase:label { 
                bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                }
        }
"""

sparql_pandas(cats4, endpoint)

## Further examples
The queries below are just for reference. They show some of other uses and syntax of Sparql. They're in roughly ascending trickiness.

In [ ]:
## Using COUNT to find the number of American journalists not born in Chicago
no_chicago = """
    SELECT (COUNT (?person) as ?personLabel)
        WHERE {
            ?person 	wdt:P106 	wd:Q1930187 ;
                        wdt:P27 	wd:Q30 .
            ?person 	wdt:P19 	?place .
            FILTER (?place != wd:Q1297)
            SERVICE wikibase:label { 
                bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". 
                }  
    }
"""
sparql_pandas(no_chicago, endpoint)

In [ ]:
# Use UNION to find Scottish poets and novelists
# And use a chain to find birthplaces that are in Scotland, not just = Scotland
writers = """
    SELECT DISTINCT ?writerLabel 
    WHERE {
        {?writer wdt:P106 wd:Q49757 .} 
        UNION 
        {?writer wdt:P106 wd:Q6625963 .}
        ?writer wdt:P19 ?place .
        ?place wdt:P131* wd:Q22 .
        SERVICE wikibase:label {
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
        }  
    }
    LIMIT 20
"""
sparql_pandas(writers, endpoint)

In [ ]:
# Use layers to make a colour-coded map of where astronauts were born
# Best seen on Wikidata where you can select map view: https://w.wiki/BSW
space = """
    SELECT ?location ?birthplaceLabel ?personLabel ?layerLabel
    WHERE {
        ?person wdt:P31 wd:Q5 ;
                wdt:P106 wd:Q11631 ;
                wdt:P19 ?birthplace ;
                wdt:P21 ?gender .
        ?birthplace wdt:P625 ?location .
        BIND(?gender AS ?layer) 
        SERVICE wikibase:label { 
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". 
        }
    }
"""
sparql_pandas(space, endpoint)

In [ ]:
# 10 largest cities with female mayors
# Use FILTER NOT EXISTS to find current mayors
# by only selecting where the job has no end date
mayors = """
    SELECT DISTINCT ?cityLabel ?population ?mayorLabel WHERE {
      ?city wdt:P31/wdt:P279* wd:Q515 ; # city or subclass of city
            wdt:P1082 ?population ; # has population
            p:P6 ?ruler .
      ?ruler ps:P6 ?mayor .
      ?mayor wdt:P39 wd:Q30185 ; # is a mayor (or leave this out to find just head of a city)
             wdt:P21 wd:Q6581072 . # is a woman
      FILTER NOT EXISTS { ?ruler pq:P582 ?x } # no end date - ie current
      SERVICE wikibase:label {
          bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
          }
    ORDER BY DESC (?population)
    LIMIT 10
"""
sparql_pandas(mayors, endpoint)